In [2]:
from utilities import *
import pandas as pd

In [3]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
d = DataProvider(filepath=datafile, route='910')

In [ ]:
d.data

In [4]:
for st, et in zip(d.data['start_time'], d.data['end_time']):
    print(Shift(d.constraints, st))

Shift(constraints=<utilities.data.Contraints object at 0x000001548E835760>, start_time=300, end_time=0, max_end_time=1020, working_time=0, short_rests=0, rest_time=0, breaks=0, break_time=0, trips=())
Shift(constraints=<utilities.data.Contraints object at 0x000001548E835760>, start_time=320, end_time=0, max_end_time=1040, working_time=0, short_rests=0, rest_time=0, breaks=0, break_time=0, trips=())
Shift(constraints=<utilities.data.Contraints object at 0x000001548E835760>, start_time=355, end_time=0, max_end_time=1075, working_time=0, short_rests=0, rest_time=0, breaks=0, break_time=0, trips=())
Shift(constraints=<utilities.data.Contraints object at 0x000001548E835760>, start_time=380, end_time=0, max_end_time=1100, working_time=0, short_rests=0, rest_time=0, breaks=0, break_time=0, trips=())
Shift(constraints=<utilities.data.Contraints object at 0x000001548E835760>, start_time=400, end_time=0, max_end_time=1120, working_time=0, short_rests=0, rest_time=0, breaks=0, break_time=0, trips